In [19]:
pip install xaif


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [69]:
pip install oamf


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


# oAMF Pipeline Execution with xAIF Data

This notebook demonstrates how to use `oAMF` for executing a pipeline and manipulating xAIF data. It includes the following steps:

1. **Importing Required Libraries**
2. **Initializing the oAMF Library**
3. **Defining File Paths**
4. **(Optional) Initializing AIF with Free Text**
5. **Defining Modules to Load**
6. **Defining Pipeline Structure**
7. **Loading and Deploying Modules**
8. **Executing the Pipeline**
9. **Extracting the Path from the Output**
10. **Reading and Loading the Predicted xAIF Output**
11. **Converting xAIF Data to DataFrame**
12. **Optional Workflow Execution**





### 1. **Import Required Libraries**




In [2]:
# Import necessary libraries for pipeline execution and data manipulation
from oamf import oAMF  # Import oAMF for pipeline execution
from xaif import AIF   # Import xaif for manipulating xAIF data

### 2. **Initialize oAMF Library**

In [3]:
# Initialize the oAMF library
oamf = oAMF()



### **5. Define Modules to Load**

In [4]:
# Define the modules to load, specifying the following for each module:
# (URL, type ['repo' or 'ws'], deployment route, tag)
# - "type" indicates whether the deployment is local ("repo") or uses an already deployed web service ("ws").
# - "repo" refers to local deployment, while "ws" refers to using an existing web service.

# List of workflow modules to load
modules_to_load = [
    ("http://default-turninator.amfws.arg.tech/turninator-01", "ws", "turninator-01", "turninator"),
    ("http://default-segmenter.amfws.arg.tech/segmenter-01", "ws", "segmenter-01", "segmenter"),
    ("http://targer.amfws.arg.tech/targer-segmenter", "ws", "targer-segmenter", "targer"),
    ("http://default-proposition-unitiser.amfws.arg.tech/propositionUnitizer-01", "ws", "propositionUnitizer-01", "propositionUnitizer"),
    ("http://vanilla-caasr.amfws.arg.tech/caasra", "ws", "caasra", "caasra"),
    ("http://cascading-propositionUnitiser.amfws.arg.tech/propositionalizer-cascading", "ws", "propositionalizer-cascading", "cascading"),
    ("http://amfws-ari.arg.tech/", "ws", "ari", "ari"),
    ("http://bert-te.amfws.arg.tech/bert-te", "ws", "bert-te", "bert-te")
]


### **6. Define Pipeline Structure**

In [5]:
# Define the pipeline structure as a directed graph.
# The pipeline graph is constructed based on the tags associated with each module
# when loading the modules. Each tuple in the graph represents a sequence of 
# module execution, where the first element is the source module and the second 
# element is the destination module. This creates a flow of data through the pipeline.

pipeline_graph = [
    ("turninator", "targer"),        # Data flows from 'turninator' to 'targer'
    ("targer", "segmenter"),         # Data flows from 'targer' to 'segmenter'
    ("segmenter", "propositionUnitizer"), # Data flows from 'segmenter' to 'propositionUnitizer'
    ("propositionUnitizer", "cascading"),  # Data flows from 'propositionUnitizer' to 'cascading'
    ("cascading", "bert-te")         # Data flows from 'cascading' to 'bert-te'
]


## **7. Load and Deploy Modules**

In [6]:
# Load and deploy only the specified modules used in constructing the pipeline.
# The modules to be loaded are determined based on the pipeline graph, which defines
# the sequence of module execution. Each module in the pipeline is automatically
# loaded and deployed for execution.

oamf.load_modules(modules_to_load)


Module 'turninator-01' of type 'ws' loaded from http://default-turninator.amfws.arg.tech/turninator-01.
Module 'segmenter-01' of type 'ws' loaded from http://default-segmenter.amfws.arg.tech/segmenter-01.
Module 'targer-segmenter' of type 'ws' loaded from http://targer.amfws.arg.tech/targer-segmenter.
Module 'propositionUnitizer-01' of type 'ws' loaded from http://default-proposition-unitiser.amfws.arg.tech/propositionUnitizer-01.
Module 'caasra' of type 'ws' loaded from http://vanilla-caasr.amfws.arg.tech/caasra.
Module 'propositionalizer-cascading' of type 'ws' loaded from http://cascading-propositionUnitiser.amfws.arg.tech/propositionalizer-cascading.
Module '' of type 'ws' loaded from http://amfws-ari.arg.tech/.
Module 'bert-te' of type 'ws' loaded from http://bert-te.amfws.arg.tech/bert-te.


{'turninator': 'turninator-01',
 'segmenter': 'segmenter-01',
 'targer': 'targer-segmenter',
 'propositionUnitizer': 'propositionUnitizer-01',
 'caasra': 'caasra',
 'cascading': 'propositionalizer-cascading',
 'ari': '',
 'bert-te': 'bert-te'}

## **8. Execute the Pipeline**

In [23]:
# Execute the pipeline using the defined pipeline graph and input file.
# The input file is in xAIF format, and the pipeline will process this file.
# The result will  be saved to a temporary path or returned as a JSON object.

# Define the input file path
input_file = "example.json"  # Path to the input xAIF data file

# Execute the pipeline with the specified graph and input file
# This returns the output path (where the result is saved) and the output data as JSON
output_path, output_json = oamf.pipelineExecutor(pipeline_graph, input_file)

# Print the resulting output JSON for further inspection
print(output_json)


Processing module 'turninator' and its dependencies...
Processing module with tag 'turninator' -> Repo: 'turninator-01'
Module 'turninator-01' is of type 'ws'. No local setup needed.
Processing module with tag 'targer' -> Repo: 'targer-segmenter'
Module 'targer-segmenter' is of type 'ws'. No local setup needed.
Processing module 'segmenter' and its dependencies...
Processing module with tag 'segmenter' -> Repo: 'segmenter-01'
Module 'segmenter-01' is of type 'ws'. No local setup needed.
Processing module with tag 'propositionUnitizer' -> Repo: 'propositionUnitizer-01'
Module 'propositionUnitizer-01' is of type 'ws'. No local setup needed.
Processing module 'targer' and its dependencies...
Processing module 'propositionUnitizer' and its dependencies...
Processing module with tag 'cascading' -> Repo: 'propositionalizer-cascading'
Module 'propositionalizer-cascading' is of type 'ws'. No local setup needed.
Processing module 'cascading' and its dependencies...
Processing module with tag 'b

## 3. **Alternatively, Use xaif to Construct xAIF from Text and Execute the Pipeline**

In [14]:
# Example: Initialize AIF with free text to generate xAIF format
import json
import tempfile
xaif_data = AIF("Sample input text.") 

# Write temporary JSON
def write_temp_json(data):
    with tempfile.NamedTemporaryFile(mode='w+', delete=False, suffix='.json') as temp_file:
        json.dump(data, temp_file, indent=4)
        temp_file_path = temp_file.name
    print(f"Temporary JSON file created at: {temp_file_path}")
    return temp_file_path
input_file_path = write_temp_json(xaif_data.xaif)  # save xAIF to a file

# Execute the pipeline with the specified graph and input file
# This returns the output path (where the result is saved) and the output data as JSON
output_path, output_json = oamf.pipelineExecutor(pipeline_graph, input_file_path)


Temporary JSON file created at: /tmp/tmp5c0my6d8.json
Processing module 'turninator' and its dependencies...
Processing module with tag 'turninator' -> Repo: 'turninator-01'
Module 'turninator-01' is of type 'ws'. No local setup needed.
Processing module with tag 'targer' -> Repo: 'targer-segmenter'
Module 'targer-segmenter' is of type 'ws'. No local setup needed.
Processing module 'segmenter' and its dependencies...
Processing module with tag 'segmenter' -> Repo: 'segmenter-01'
Module 'segmenter-01' is of type 'ws'. No local setup needed.
Processing module with tag 'propositionUnitizer' -> Repo: 'propositionUnitizer-01'
Module 'propositionUnitizer-01' is of type 'ws'. No local setup needed.
Processing module 'targer' and its dependencies...
Processing module 'propositionUnitizer' and its dependencies...
Processing module with tag 'cascading' -> Repo: 'propositionalizer-cascading'
Module 'propositionalizer-cascading' is of type 'ws'. No local setup needed.
Processing module 'cascading'

## **Run Pipeline using n8n Workflow**

In [17]:
# Override the manually defined pipeline with one built using n8n (if applicable). Build workflow on n8n and download it and excute throught the oAMF API
# Define the workflow file path
workflow_file = "workflow.json"  # Path to the input xAIF data file
oamf.pipelineExecutor(pipeline_graph, input_file_path, workflow_file)

Execution Order: ['Set Dialog Text', 'Text to JSON2', 'JSON to File3', 'Turninator2', 'Segmenter2', 'Propositionalizer1', 'bert-te1']
Dependencies: [('Text to JSON2', 'JSON to File3'), ('JSON to File3', 'Turninator2'), ('Set Dialog Text', 'Text to JSON2'), ('Turninator2', 'Segmenter2'), ('Segmenter2', 'Propositionalizer1'), ('Propositionalizer1', 'bert-te1')]
Response from Turninator2 (HTTP POST http://default-turninator.amfws.arg.tech/turninator-01): 200
{"AIF": {"nodes": [{"text": " Sample input text..", "type": "L", "nodeID": 0}], "edges": [], "locutions": [{"personID": 1, "nodeID": 0}], "schemefulfillments": [], "descriptorfulfillments": [], "participants": [{"participantID": 1, "firstname": "Default", "surname": "Speaker"}]}, "ova": [], "dialog": {}, "text": {"txt": " Default Speaker <span class=\"highlighted\" id=\"0\"> Sample input text..</span>.<br><br>"}}
Response from Segmenter2 (HTTP POST http://default-segmenter.amfws.arg.tech/segmenter-01): 200
{"AIF":{"descriptorfulfillme

"Pipeline execution completed. Final result: {'Turninator2': {'file': ('response.json', <_io.BufferedReader name='/tmp/tmpemf7fp38'>)}, 'Segmenter2': {'file': ('response.json', <_io.BufferedReader name='/tmp/tmp_a2y4bkc'>)}, 'Propositionalizer1': {'file': ('response.json', <_io.BufferedReader name='/tmp/tmpy9_2n7qm'>)}, 'bert-te1': {'file': ('response.json', <_io.BufferedReader name='/tmp/tmpau6zteia'>)}}"

## **Use xaif library to extract and export the xaif to csv**

In [22]:
# Export to dataframe in CSV format
xaif = AIF(output_json) # initialise xaif kibrary with xaif data obtained from the pipeline output
predection_dataframe = xaif.get_csv("argument-relation")
print(predection_dataframe)


Empty DataFrame
Columns: [proposition1_id, proposition1_text, proposition2_id, proposition2_text, relation]
Index: []
